# **Data Collection and Cleaning**

In [ ]:
import pandas as pd
import numpy as np
import os
# Import gdown to download files from Google Drive
!pip install gdown --quiet
import gdown
from google.colab import drive

# Mount Google Drive
print("--- Mounting Google Drive ---")
drive.mount('/content/drive')
print("--- Google Drive Mounted ---")

# --- Konfigurasi Path File ---
# Google Drive file IDs extracted from the shared links
file_ids = {
    'yield.csv': '1P2Z1zzxc73OwnqwTkS_vXqzQy4SpmwRI',
    'rainfall.csv': '1ZejlVom6mxw8lgFZbY9KSMs0v_gtwApg',
    'pesticides.csv': '1o15vQGsopHLPcDGUJ75ll5khWlZV7BnX',
    'temp.csv': '1hQ7waGIDF-k-5g62NQtFaYomygk4bXfo'
}

# Directory to save the downloaded files and output files in Google Drive
# PLEASE VERIFY AND UPDATE THIS PATH TO MATCH YOUR GOOGLE DRIVE FOLDER STRUCTURE
# The path below assumes the folder with ID '1x4pIBNhB9oLVkaBvXsoOPTnCGdv6KcCx' is directly in 'My Drive'
# If the folder name is different, or it's nested within other folders, adjust the path accordingly.
base_data_path = '/content/drive/My Drive/1x4pIBNhB9oLVkaBvXsoOPTnCGdv6KcCx'
os.makedirs(base_data_path, exist_ok=True)


# Download files from Google Drive to the specified path
print("\n--- Downloading Data Files from Google Drive ---")
for filename, file_id in file_ids.items():
    output_path = os.path.join(base_data_path, filename)
    # Construct the Google Drive download URL
    url = f'https://drive.google.com/uc?id={file_id}'
    try:
        gdown.download(url, output_path, quiet=True)
        print(f"  Downloaded {filename} to {output_path}")
    except Exception as e:
        print(f"  Error downloading {filename}: {e}")
        # Exit if a file fails to download
        exit()

# Tentukan folder untuk menyimpan data yang telah dipisahkan per jenis panen
per_crop_output_folder = os.path.join(base_data_path, 'per_crop_data')
os.makedirs(per_crop_output_folder, exist_ok=True) # Buat folder jika belum ada

print("\n--- Memulai Proses Pengolahan Data Hasil Panen ---")

# --- Bagian 1: Penggabungan dan Pemisahan Data ---
print("\n[Langkah 1/3] Memuat dan Menggabungkan Data Mentah...")

# Path ke file CSV mentah (now local paths after download to Google Drive)
yield_file = os.path.join(base_data_path, 'yield.csv')
rainfall_file = os.path.join(base_data_path, 'rainfall.csv')
pesticides_file = os.path.join(base_data_path, 'pesticides.csv')
temp_file = os.path.join(base_data_path, 'temp.csv')

# Memuat setiap file CSV ke DataFrame
try:
    # Added engine='python' as a potential workaround for tokenization errors,
    # although the primary issue is likely the file path.
    # Removed engine='python' as it can be slower and the file path issue is resolved.
    # If ParserError persists, consider adding engine='python' back or inspecting the file content.
    df_yield = pd.read_csv(yield_file)
    df_rainfall = pd.read_csv(rainfall_file)
    df_pesticides = pd.read_csv(pesticides_file)
    df_temp = pd.read_csv(temp_file)
    print("  Semua file CSV mentah berhasil dimuat.")
except FileNotFoundError as e:
    print(f"  Error: Salah satu file mentah tidak ditemukan. Pastikan path sudah benar. Detail: {e}")
    print("  Please ensure the files were downloaded correctly to the 'base_data_path'.")
    exit()
except pd.errors.ParserError as e:
    print(f"  Error: Failed to parse one of the CSV files. This could be due to incorrect file format, delimiter, or structure.")
    print(f"  Detail: {e}")
    print(f"  Please inspect the problematic file around the indicated line number to check for inconsistencies.")
    print(f"  You might need to specify the correct 'sep' (delimiter) parameter in pd.read_csv if it's not comma-separated.")
    exit()
except Exception as e:
    print(f"  An unexpected error occurred during file loading: {e}")
    exit()


# Fungsi untuk menstandardisasi nama kolom
def standardize_columns(df, column_mapping):
    df.columns = df.columns.str.strip() # Hapus spasi di awal/akhir nama kolom
    new_names = {old_name: new_name for old_name, new_name in column_mapping.items() if old_name in df.columns}
    if new_names:
        df.rename(columns=new_names, inplace=True)
    return df

# Pemetaan nama kolom untuk standardisasi
col_map_yield = {'Area': 'Country', 'Item': 'Crop_Item'}
col_map_rainfall = {'Area': 'Country', 'Year': 'Year', 'average_rain_fall_mm_per_year': 'Avg_Rainfall_mm'}
col_map_pesticides = {'Area': 'Country', 'Item': 'Crop_Item'}
col_map_temp = {'country': 'Country', 'year': 'Year', 'avg_temp': 'Avg_Temperature_celsius'}

# Terapkan standardisasi kolom
df_yield = standardize_columns(df_yield, col_map_yield)
df_rainfall = standardize_columns(df_rainfall, col_map_rainfall)
df_pesticides = standardize_columns(df_pesticides, col_map_pesticides)
df_temp = standardize_columns(df_temp, col_map_temp)
print("  Nama kolom telah disesuaikan.")

# Kunci untuk penggabungan DataFrame
merge_keys_common = ['Country', 'Year']

# Menggabungkan DataFrame secara berurutan
print("  Melakukan penggabungan DataFrame...")
# Merged df_yield and df_pesticides first based on common keys
merged_df = pd.merge(df_yield, df_pesticides, on=merge_keys_common, how='left', suffixes=('_yield', '_pesticide'))
# Then merge with df_rainfall
merged_df = pd.merge(merged_df, df_rainfall, on=merge_keys_common, how='left')
# Finally merge with df_temp
merged_df = pd.merge(merged_df, df_temp, on=merge_keys_common, how='left')
print(f"  Bentuk DataFrame setelah penggabungan: {merged_df.shape}")

# Mengganti nama kolom hasil penggabungan agar lebih jelas
merged_df.rename(columns={
    'Value': 'Yield_Value', # Kolom 'Value' dari df_yield
    'Value_y': 'Pesticide_Value', # Kolom 'Value' dari df_pesticides - corrected from Value_x based on suffix
    'Domain_yield': 'Yield_Domain',
    'Element_yield': 'Yield_Element',
    'Unit_yield': 'Yield_Unit',
    'Domain_pesticide': 'Pesticide_Domain',
    'Element_pesticide': 'Pesticide_Element',
    'Unit_pesticide': 'Pesticide_Unit'
}, inplace=True)

# Menghapus kolom 'Crop_Item' yang duplikat dari pesticides jika ada
# Corrected the logic to check for the suffixed column name
if 'Crop_Item_pesticide' in merged_df.columns:
    merged_df.drop(columns=['Crop_Item_pesticide'], inplace=True)
# Mengganti nama kolom 'Crop_Item_yield' menjadi 'Crop_Item'
if 'Crop_Item_yield' in merged_df.columns:
    merged_df.rename(columns={'Crop_Item_yield': 'Crop_Item'}, inplace=True)


# Menghapus duplikat baris
merged_df.drop_duplicates(inplace=True)

# Konversi kolom 'Year' ke integer dan tangani nilai yang hilang
merged_df['Year'] = pd.to_numeric(merged_df['Year'], errors='coerce').astype('Int64')
merged_df.dropna(subset=['Year'], inplace=True)
merged_df['Year'] = merged_df['Year'].astype(int)

# Mengisi nilai yang hilang (NaN) dengan rata-rata kolom numerik
for col in merged_df.columns:
    # Check if the column is numeric before attempting to fillna with mean
    if merged_df[col].dtype in ['float64', 'int64'] and col not in ['Country', 'Crop_Item', 'Year']:
         if merged_df[col].isnull().any():
            merged_df[col].fillna(merged_df[col].mean(), inplace=True)

print("  Pembersihan dan persiapan akhir DataFrame gabungan selesai.")

# Simpan DataFrame hasil gabungan (opsional, untuk inspeksi)
# Ensure the output path is writable
output_combined_csv = os.path.join(base_data_path, 'combined_crop_yield_data.csv')
try:
    merged_df.to_csv(output_combined_csv, index=False)
    print(f"  DataFrame gabungan awal berhasil disimpan ke: {output_combined_csv}")
except Exception as e:
    print(f"  Error saving combined DataFrame: {e}")


print("\n[Langkah 2/3] Memisahkan Data Per Jenis Panen dan Menyimpannya...")

# Daftar jenis panen yang ingin dipisahkan
target_crop_items = [
    'Maize', 'Potatoes', 'Rice, paddy', 'Sorghum', 'Soybeans', 'Wheat'
]

# Iterasi untuk memisahkan dan menyimpan data per jenis panen
for crop_name in target_crop_items:
    df_crop_specific = merged_df[merged_df['Crop_Item'] == crop_name].copy()

    # Buat nama file yang bersih (mengganti spasi dan koma)
    sanitized_crop_name = crop_name.replace(" ", "_").replace(",", "-")
    output_filename = f"combined_{sanitized_crop_name}_crop_yield_data.csv"
    output_filepath = os.path.join(per_crop_output_folder, output_filename)

    if not df_crop_specific.empty:
        try:
            df_crop_specific.to_csv(output_filepath, index=False)
            print(f"  Berhasil menyimpan {df_crop_specific.shape[0]} baris untuk '{crop_name}' ke: {os.path.basename(output_filepath)}")
        except Exception as e:
            print(f"  Error saving data for '{crop_name}': {e}")
    else:
        print(f"  Tidak ada data untuk '{crop_name}'. File tidak dibuat.")

print("  Proses pemisahan data selesai.")

# --- Bagian 2: Penghapusan Kolom pada Dataset Terpisah ---
print("\n[Langkah 3/3] Menghapus Kolom yang Tidak Diperlukan dari Dataset Terpisah...")

# Daftar kolom yang akan dihapus dari setiap file
columns_to_drop = [
    'Domain Code', 'Yield_Domain', 'Pesticide_Domain',
    'Element Code', 'Yield_Element', 'Pesticide_Element',
    'Item Code', 'Year Code', 'Yield_Unit', 'Pesticide_Unit'
]

# Iterasi melalui setiap file yang baru saja dipisahkan dan hapus kolomnya
for crop_name in target_crop_items:
    sanitized_crop_name = crop_name.replace(" ", "_").replace(",", "-")
    file_path = os.path.join(per_crop_output_folder, f"combined_{sanitized_crop_name}_crop_yield_data.csv")

    if os.path.exists(file_path):
        try:
            df_crop = pd.read_csv(file_path)
            initial_columns = df_crop.columns.tolist()

            # Hapus kolom-kolom yang ada di daftar
            df_crop.drop(columns=columns_to_drop, errors='ignore', inplace=True)

            # Periksa apakah ada kolom yang benar-benar dihapus
            columns_deleted_in_this_file = [col for col in columns_to_drop if col in initial_columns and col not in df_crop.columns]

            if columns_deleted_in_this_file:
                print(f"  File '{os.path.basename(file_path)}': Kolom dihapus: {columns_deleted_in_this_file}")
                df_crop.to_csv(file_path, index=False)
                print(f"  Perubahan disimpan kembali ke: {os.path.basename(file_path)}")
            else:
                print(f"  File '{os.path.basename(file_path)}': Tidak ada kolom dari daftar yang ditemukan untuk dihapus.")

        except pd.errors.EmptyDataError:
            print(f"  Peringatan: File '{os.path.basename(file_path)}' kosong. Melewati penghapusan kolom.")
        except Exception as e:
            print(f"  Error saat memproses '{os.path.basename(file_path)}': {e}")
    else:
        print(f"  File '{os.path.basename(file_path)}' tidak ditemukan. Melewati penghapusan kolom.")

print("\n--- Proses Penghapusan Kolom Selesai ---")
print("\nSemua proses pengolahan data (Penggabungan, Pemisahan, dan Penghapusan Kolom) telah selesai.")

--- Mounting Google Drive ---
Mounted at /content/drive
--- Google Drive Mounted ---

--- Downloading Data Files from Google Drive ---
  Downloaded yield.csv to /content/drive/My Drive/1x4pIBNhB9oLVkaBvXsoOPTnCGdv6KcCx/yield.csv
  Downloaded rainfall.csv to /content/drive/My Drive/1x4pIBNhB9oLVkaBvXsoOPTnCGdv6KcCx/rainfall.csv
  Downloaded pesticides.csv to /content/drive/My Drive/1x4pIBNhB9oLVkaBvXsoOPTnCGdv6KcCx/pesticides.csv
  Downloaded temp.csv to /content/drive/My Drive/1x4pIBNhB9oLVkaBvXsoOPTnCGdv6KcCx/temp.csv

--- Memulai Proses Pengolahan Data Hasil Panen ---

[Langkah 1/3] Memuat dan Menggabungkan Data Mentah...
  Semua file CSV mentah berhasil dimuat.
  Nama kolom telah disesuaikan.
  Melakukan penggabungan DataFrame...
  Bentuk DataFrame setelah penggabungan: (109366, 19)


/tmp/ipython-input-1-3283368530.py:154: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[col].fillna(merged_df[col].mean(), inplace=True)


  Pembersihan dan persiapan akhir DataFrame gabungan selesai.
  DataFrame gabungan awal berhasil disimpan ke: /content/drive/My Drive/1x4pIBNhB9oLVkaBvXsoOPTnCGdv6KcCx/combined_crop_yield_data.csv

[Langkah 2/3] Memisahkan Data Per Jenis Panen dan Menyimpannya...
  Berhasil menyimpan 14731 baris untuk 'Maize' ke: combined_Maize_crop_yield_data.csv
  Berhasil menyimpan 14100 baris untuk 'Potatoes' ke: combined_Potatoes_crop_yield_data.csv
  Berhasil menyimpan 12072 baris untuk 'Rice, paddy' ke: combined_Rice-_paddy_crop_yield_data.csv
  Berhasil menyimpan 10444 baris untuk 'Sorghum' ke: combined_Sorghum_crop_yield_data.csv
  Berhasil menyimpan 9942 baris untuk 'Soybeans' ke: combined_Soybeans_crop_yield_data.csv
  Berhasil menyimpan 11964 baris untuk 'Wheat' ke: combined_Wheat_crop_yield_data.csv
  Proses pemisahan data selesai.

[Langkah 3/3] Menghapus Kolom yang Tidak Diperlukan dari Dataset Terpisah...
  File 'combined_Maize_crop_yield_data.csv': Kolom dihapus: ['Domain Code', 'Yield